# The Cancer Gene Atlas Cancer (TCGA) - Kidney Renal Papillary Cell Carcinoma (KIRP)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
IPython 5.3.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.13.1
pandas 0.21.0
clustergrammer_widget 1.9.0
seaborn 0.7.1
Fri Apr 27 15:34:59 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Kidney renal papillary cell carcinoma [KIRP]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-2K-A9WE,8003,KIRP,TCGA,73292C19-D6A8-4BC4-97BC-CCCE54F264F8,2K,53.0,NaN,398,Nationwide Children's Hospital,M0,...,27.7,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-2Z-A9JE,8004,KIRP,TCGA,C20B1007-0111-4EF6-9DF5-14DA6094FCA1,2Z,59.0,NaN,432,Nationwide Children's Hospital,M0,...,26.6,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-2Z-A9JL,8005,KIRP,TCGA,F9C593D1-F204-4026-8A9B-A6AF0922C885,2Z,47.0,NaN,432,Nationwide Children's Hospital,M0,...,25.6,NaN,NaN,"Yes, History of Synchronous/Bilateral Malignancy",Kidney,"Other, specify",Synchronous Malignancy,2007.0,NaN,NaN
TCGA-2Z-A9JD,8006,KIRP,TCGA,CFF68090-09DF-492B-874C-0CAEB29F9361,2Z,66.0,NaN,432,Nationwide Children's Hospital,M0,...,43.4,NaN,NaN,"Yes, History of Prior Malignancy",Kidney,Clear Cell Adenocarcinoma,Prior Malignancy,1981.0,NaN,1961.0
TCGA-2Z-A9JI,8007,KIRP,TCGA,3ABD0691-F575-44CE-A13E-8F6BCEB8CABF,2Z,38.0,NaN,432,Nationwide Children's Hospital,M0,...,27.7,NaN,NaN,No,NaN,NaN,NaN,2007.0,NaN,1984.0


In [16]:
sample_meta.shape

(291, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_KIRP_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,4a863729-23fd-4d2f-9ff5-e28b43c67de9,4e882177-14e9-439e-8a1b-f72f740557b1,4046c73b-aded-474e-8e1a-8d66b1f66638,6ea48a9f-6ed2-42f1-b4a0-82876b4a7d25,1a9da861-b4f4-4707-afa4-df9d7e471f11,5a41e7b5-bbee-4bfd-9552-0d6a7e14780d,61013555-7ee6-428c-8b9b-053643b860f2,3a51726d-a7ec-4954-81bf-3433866b9dab,24d30ea7-900a-40ee-a20f-573115591401,8afd9b54-c071-42b0-84c2-7c139d120cfe,...,56ae6931-51b5-4fb1-a55e-7cbc092127fe,a8fb5e5d-9798-4d20-b65e-e063ff25d45a,da393a47-fc4e-4341-ab9c-ec2e96e07c4a,66e6cdc4-1426-474c-bdeb-874630985220,9679ee00-f6f1-4c1d-b047-5010085d830f,952b0b82-1f6c-40b9-b4cc-d6eb802ed1fc,65fdfba1-a9b6-4ae2-a412-2e2083914b8b,c80bb4c4-f75f-44d0-8225-7b37bc10fd4e,97a80aa6-f9b4-4351-ae7c-d6405f4ab3a1,ac8c0c1d-002e-48db-b6b8-e60ca10493dd
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,91.6110,56.165000,269.7540,227.48300,71.584700,127.65800,122.667000,235.72100,211.805000,111.02100,...,185.336000,116.197000,124.353000,384.44700,68.571300,139.335000,162.92200,106.829000,226.069000,251.724000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,24.7737,37.007200,57.9751,50.06380,17.783400,73.03090,108.815000,36.98870,53.423400,48.20090,...,4.290800,21.765000,21.120400,7.88351,20.601400,14.489000,8.06617,41.574600,26.875400,0.000006
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,30.3731,27.830700,42.8779,37.88920,34.916000,26.36130,11.428900,42.45080,55.661400,38.75620,...,21.963400,46.303500,21.122900,30.39700,70.039700,39.282400,20.22130,31.548600,38.361000,16.853600
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,24.9506,42.992800,36.1817,31.00460,15.402300,31.25630,64.345800,75.75110,43.347000,27.43520,...,12.195200,57.615000,44.951000,29.21280,32.183200,56.800700,19.63350,23.273700,25.265900,31.612600
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.0000,0.042659,1.4777,0.52245,0.637467,2.04735,0.787245,1.10541,0.496122,1.31819,...,0.043368,0.063259,0.504815,0.00000,0.146321,0.930197,0.00000,0.350288,0.244069,0.000000


In [19]:
matrix.shape

(199169, 323)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,4a863729-23fd-4d2f-9ff5-e28b43c67de9,4e882177-14e9-439e-8a1b-f72f740557b1,4046c73b-aded-474e-8e1a-8d66b1f66638,6ea48a9f-6ed2-42f1-b4a0-82876b4a7d25,1a9da861-b4f4-4707-afa4-df9d7e471f11,5a41e7b5-bbee-4bfd-9552-0d6a7e14780d,61013555-7ee6-428c-8b9b-053643b860f2,3a51726d-a7ec-4954-81bf-3433866b9dab,24d30ea7-900a-40ee-a20f-573115591401,8afd9b54-c071-42b0-84c2-7c139d120cfe,...,56ae6931-51b5-4fb1-a55e-7cbc092127fe,a8fb5e5d-9798-4d20-b65e-e063ff25d45a,da393a47-fc4e-4341-ab9c-ec2e96e07c4a,66e6cdc4-1426-474c-bdeb-874630985220,9679ee00-f6f1-4c1d-b047-5010085d830f,952b0b82-1f6c-40b9-b4cc-d6eb802ed1fc,65fdfba1-a9b6-4ae2-a412-2e2083914b8b,c80bb4c4-f75f-44d0-8225-7b37bc10fd4e,97a80aa6-f9b4-4351-ae7c-d6405f4ab3a1,ac8c0c1d-002e-48db-b6b8-e60ca10493dd
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,91.6110,56.165000,269.7540,227.48300,71.584700,127.65800,122.667000,235.72100,211.805000,111.02100,...,185.336000,116.197000,124.353000,384.44700,68.571300,139.335000,162.92200,106.829000,226.069000,251.724000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,24.7737,37.007200,57.9751,50.06380,17.783400,73.03090,108.815000,36.98870,53.423400,48.20090,...,4.290800,21.765000,21.120400,7.88351,20.601400,14.489000,8.06617,41.574600,26.875400,0.000006
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,30.3731,27.830700,42.8779,37.88920,34.916000,26.36130,11.428900,42.45080,55.661400,38.75620,...,21.963400,46.303500,21.122900,30.39700,70.039700,39.282400,20.22130,31.548600,38.361000,16.853600
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,24.9506,42.992800,36.1817,31.00460,15.402300,31.25630,64.345800,75.75110,43.347000,27.43520,...,12.195200,57.615000,44.951000,29.21280,32.183200,56.800700,19.63350,23.273700,25.265900,31.612600
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.0000,0.042659,1.4777,0.52245,0.637467,2.04735,0.787245,1.10541,0.496122,1.31819,...,0.043368,0.063259,0.504815,0.00000,0.146321,0.930197,0.00000,0.350288,0.244069,0.000000


In [22]:
matrix.shape

(79930, 323)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-A4-A48D-01,TCGA-A4-8518-01,TCGA-MH-A55W-01,TCGA-IZ-8196-01,TCGA-IZ-8195-01,TCGA-B3-8121-01,TCGA-J7-8537-01,TCGA-GL-8500-01,TCGA-A4-8516-01,TCGA-A4-8517-01,...,TCGA-5P-A9KF-01,TCGA-P4-AAVK-01,TCGA-2Z-A9J9-01,TCGA-P4-AAVL-01,TCGA-5P-A9KH-01,TCGA-2Z-A9JG-01,TCGA-2Z-A9JL-01,TCGA-5P-A9KA-01,TCGA-UZ-A9PK-01,TCGA-2Z-A9JQ-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,91.6110,56.165000,269.7540,227.48300,71.584700,127.65800,122.667000,235.72100,211.805000,111.02100,...,185.336000,116.197000,124.353000,384.44700,68.571300,139.335000,162.92200,106.829000,226.069000,251.724000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,24.7737,37.007200,57.9751,50.06380,17.783400,73.03090,108.815000,36.98870,53.423400,48.20090,...,4.290800,21.765000,21.120400,7.88351,20.601400,14.489000,8.06617,41.574600,26.875400,0.000006
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,30.3731,27.830700,42.8779,37.88920,34.916000,26.36130,11.428900,42.45080,55.661400,38.75620,...,21.963400,46.303500,21.122900,30.39700,70.039700,39.282400,20.22130,31.548600,38.361000,16.853600
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,24.9506,42.992800,36.1817,31.00460,15.402300,31.25630,64.345800,75.75110,43.347000,27.43520,...,12.195200,57.615000,44.951000,29.21280,32.183200,56.800700,19.63350,23.273700,25.265900,31.612600
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.0000,0.042659,1.4777,0.52245,0.637467,2.04735,0.787245,1.10541,0.496122,1.31819,...,0.043368,0.063259,0.504815,0.00000,0.146321,0.930197,0.00000,0.350288,0.244069,0.000000


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-A4-A48D-01,TCGA-A4-8518-01,TCGA-MH-A55W-01,TCGA-IZ-8196-01,TCGA-IZ-8195-01,TCGA-B3-8121-01,TCGA-J7-8537-01,TCGA-GL-8500-01,TCGA-A4-8516-01,TCGA-A4-8517-01,...,TCGA-5P-A9KF-01,TCGA-P4-AAVK-01,TCGA-2Z-A9J9-01,TCGA-P4-AAVL-01,TCGA-5P-A9KH-01,TCGA-2Z-A9JG-01,TCGA-2Z-A9JL-01,TCGA-5P-A9KA-01,TCGA-UZ-A9PK-01,TCGA-2Z-A9JQ-01
ARF5,91.6110,56.165000,269.7540,227.48300,71.584700,127.65800,122.667000,235.72100,211.805000,111.02100,...,185.336000,116.197000,124.353000,384.44700,68.571300,139.335000,162.92200,106.829000,226.069000,251.724000
M6PR,24.7737,37.007200,57.9751,50.06380,17.783400,73.03090,108.815000,36.98870,53.423400,48.20090,...,4.290800,21.765000,21.120400,7.88351,20.601400,14.489000,8.06617,41.574600,26.875400,0.000006
ESRRA,30.3731,27.830700,42.8779,37.88920,34.916000,26.36130,11.428900,42.45080,55.661400,38.75620,...,21.963400,46.303500,21.122900,30.39700,70.039700,39.282400,20.22130,31.548600,38.361000,16.853600
FKBP4,24.9506,42.992800,36.1817,31.00460,15.402300,31.25630,64.345800,75.75110,43.347000,27.43520,...,12.195200,57.615000,44.951000,29.21280,32.183200,56.800700,19.63350,23.273700,25.265900,31.612600
CYP26B1,0.0000,0.042659,1.4777,0.52245,0.637467,2.04735,0.787245,1.10541,0.496122,1.31819,...,0.043368,0.063259,0.504815,0.00000,0.146321,0.930197,0.00000,0.350288,0.244069,0.000000


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_kirp_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [31]:
normalized_matrix = matrix.copy()

In [32]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining mssing values to matrix mean

In [33]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [34]:
normalized_matrix.head()

,TCGA-A4-A48D-01,TCGA-A4-8518-01,TCGA-MH-A55W-01,TCGA-IZ-8196-01,TCGA-IZ-8195-01,TCGA-B3-8121-01,TCGA-J7-8537-01,TCGA-GL-8500-01,TCGA-A4-8516-01,TCGA-A4-8517-01,...,TCGA-5P-A9KF-01,TCGA-P4-AAVK-01,TCGA-2Z-A9J9-01,TCGA-P4-AAVL-01,TCGA-5P-A9KH-01,TCGA-2Z-A9JG-01,TCGA-2Z-A9JL-01,TCGA-5P-A9KA-01,TCGA-UZ-A9PK-01,TCGA-2Z-A9JQ-01
A1BG,0.000158,0.015026,0.080231,0.049639,0.056189,0.080231,0.080231,0.080231,0.080231,0.080231,...,0.077021,0.080231,0.080231,0.080231,0.080231,0.080231,0.091809,0.038269,0.080231,0.080231
A1CF,0.493945,2.292100,1.373910,0.150753,0.010448,0.348171,1.006010,1.284460,5.485700,0.313398,...,1.379090,0.228032,0.013284,0.205204,0.011219,0.221918,0.426893,0.194820,0.192328,0.068292
A2M,75.839900,41.588800,36.185300,107.511000,30.488000,146.283000,110.950000,23.026200,76.782600,66.764800,...,40.714100,50.502600,376.100000,38.338300,77.771000,48.820800,75.441300,64.106900,65.889800,31.707700
A2ML1,0.030624,0.019224,0.104063,0.176705,0.129467,0.118374,0.061833,0.036969,0.052842,0.065287,...,0.119663,0.051708,0.037874,0.113706,0.077587,0.062333,0.153980,0.107630,0.044690,0.060592
A3GALT2,0.051440,0.020415,0.049315,0.034285,0.051440,0.051440,0.051440,0.051440,0.024240,0.019400,...,0.018533,0.029967,0.051440,0.033997,0.014204,0.017816,0.022683,0.051440,0.021533,0.051440


In [35]:
normalized_matrix.shape

(19062, 323)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [36]:
samples = normalized_matrix.columns.tolist()

In [37]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [38]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [39]:
normalized_matrix.columns = samples

In [40]:
normalized_matrix.head()

,TCGA-A4-A48D-01,TCGA-A4-8518-01,TCGA-MH-A55W-01,TCGA-IZ-8196-01,TCGA-IZ-8195-01,TCGA-B3-8121-01,TCGA-J7-8537-01,TCGA-GL-8500-01,TCGA-A4-8516-01,TCGA-A4-8517-01,...,TCGA-5P-A9KF-01,TCGA-P4-AAVK-01,TCGA-2Z-A9J9-01,TCGA-P4-AAVL-01,TCGA-5P-A9KH-01,TCGA-2Z-A9JG-01,TCGA-2Z-A9JL-01,TCGA-5P-A9KA-01,TCGA-UZ-A9PK-01,TCGA-2Z-A9JQ-01
A1BG,-3.691535,-1.714571,-0.933269,-1.102171,-1.117452,-0.873514,-0.884866,-0.927972,-0.916225,-0.948807,...,-0.863035,-0.900732,-0.846215,-0.849271,-0.916348,-0.839662,-0.706346,-1.131403,-0.921234,-0.836571
A1CF,-0.196261,0.468805,0.300346,-0.619728,-1.848051,-0.236065,0.213393,0.276405,0.918664,-0.357054,...,0.389950,-0.447079,-1.627232,-0.441428,-1.770729,-0.397812,-0.038914,-0.424618,-0.541535,-0.906543
A2M,1.989958,1.727548,1.720920,2.233459,1.617029,2.387336,2.255918,1.529907,2.064695,1.971398,...,1.860102,1.898239,2.824745,1.830019,2.070127,1.944600,2.208376,2.092653,1.993241,1.760250
A2ML1,-1.403879,-1.607585,-0.820316,-0.550745,-0.754941,-0.704601,-0.997992,-1.264483,-1.097592,-1.038324,...,-0.671683,-1.091516,-1.172211,-0.697831,-0.930901,-0.949288,-0.481768,-0.682319,-1.175367,-0.958498
A3GALT2,-1.178639,-1.581469,-1.144634,-1.262891,-1.155800,-1.066557,-1.077909,-1.121015,-1.436039,-1.565346,...,-1.481688,-1.328435,-1.039258,-1.222171,-1.668283,-1.493187,-1.313540,-1.002952,-1.492471,-1.029614


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [41]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [42]:
normalized_matrix.head()

,TCGA-A4-A48D-01,TCGA-A4-8518-01,TCGA-MH-A55W-01,TCGA-IZ-8196-01,TCGA-IZ-8195-01,TCGA-B3-8121-01,TCGA-J7-8537-01,TCGA-GL-8500-01,TCGA-A4-8516-01,TCGA-A4-8517-01,...,TCGA-5P-A9KF-01,TCGA-P4-AAVK-01,TCGA-2Z-A9J9-01,TCGA-P4-AAVL-01,TCGA-5P-A9KH-01,TCGA-2Z-A9JG-01,TCGA-2Z-A9JL-01,TCGA-5P-A9KA-01,TCGA-UZ-A9PK-01,TCGA-2Z-A9JQ-01
A1BG,-2.826032,-1.532305,-1.004428,-1.198473,-0.928974,-1.031779,-0.939811,-0.841600,-0.887604,-0.802731,...,-0.894709,-0.781919,-0.971180,-0.958330,-0.787248,-0.869305,-0.893171,-1.289910,-0.909365,-0.800960
A1CF,0.151880,0.777030,0.198458,-0.737364,-1.664600,-0.463051,0.039010,0.316305,0.866136,-0.214834,...,0.366991,-0.310685,-1.722495,-0.545731,-1.653093,-0.432457,-0.249739,-0.602872,-0.555578,-0.895477
A2M,2.116362,1.900833,1.656655,2.132847,1.832357,2.346610,2.130376,1.497190,2.002952,2.073712,...,1.849281,1.913802,2.758751,1.759214,2.190950,1.857668,2.149144,2.026455,1.928148,1.795105
A2ML1,-1.218590,-1.422852,-0.897282,-0.674580,-0.508424,-0.889387,-1.052054,-1.181314,-1.083208,-0.906097,...,-0.701505,-0.981057,-1.282487,-0.804264,-0.802001,-0.978525,-0.688169,-0.854035,-1.168357,-0.959104
A3GALT2,-0.976298,-1.401713,-1.220117,-1.380883,-0.975533,-1.218590,-1.124676,-1.033367,-1.435447,-1.452208,...,-1.500337,-1.236582,-1.154123,-1.358925,-1.555471,-1.526841,-1.514999,-1.160222,-1.494709,-1.041633


# Normalize Matrix (z-score the rows)

In [43]:
uf.zscore(normalized_matrix, 'row')

In [44]:
normalized_matrix.head()

,TCGA-A4-A48D-01,TCGA-A4-8518-01,TCGA-MH-A55W-01,TCGA-IZ-8196-01,TCGA-IZ-8195-01,TCGA-B3-8121-01,TCGA-J7-8537-01,TCGA-GL-8500-01,TCGA-A4-8516-01,TCGA-A4-8517-01,...,TCGA-5P-A9KF-01,TCGA-P4-AAVK-01,TCGA-2Z-A9J9-01,TCGA-P4-AAVL-01,TCGA-5P-A9KH-01,TCGA-2Z-A9JG-01,TCGA-2Z-A9JL-01,TCGA-5P-A9KA-01,TCGA-UZ-A9PK-01,TCGA-2Z-A9JQ-01
A1BG,-15.091039,-4.811559,-0.617239,-2.159047,-0.017713,-0.834560,-0.103820,0.676535,0.311002,0.985369,...,0.254547,1.150734,-0.353066,-0.250962,1.108395,0.456395,0.266769,-2.885574,0.138097,0.999443
A1CF,0.580074,1.316652,0.634955,-0.467672,-1.560182,-0.144465,0.447086,0.773807,1.421641,0.147996,...,0.833528,0.035059,-1.628397,-0.241882,-1.546624,-0.108417,0.106869,-0.309208,-0.253484,-0.653968
A2M,0.207888,-0.371241,-1.027350,0.252183,-0.555236,0.826568,0.245544,-1.455835,-0.096844,0.093288,...,-0.509763,-0.336394,1.933995,-0.751772,0.408308,-0.487225,0.295974,-0.033691,-0.297844,-0.655332
A2ML1,-1.265459,-2.053613,-0.025678,0.833629,1.474750,0.004787,-0.622873,-1.121627,-0.743081,-0.059688,...,0.729740,-0.348926,-1.512008,0.333238,0.341969,-0.339156,0.781198,0.141195,-1.071632,-0.264218
A3GALT2,0.886360,-1.595061,-0.535820,-1.473558,0.890821,-0.526914,0.020878,0.553482,-1.791827,-1.889590,...,-2.170326,-0.631857,-0.150880,-1.345480,-2.491918,-2.324924,-2.255849,-0.186455,-2.137496,0.505262


# Merge Like Column (by taking the mean)

In [45]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [46]:
normalized_matrix.shape

(19062, 323)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [47]:
uf.mapgenesymbols(normalized_matrix)

In [48]:
normalized_matrix.shape

(18635, 323)

# Merge Duplicate Genes By Rows

In [49]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [50]:
normalized_matrix.shape

(18546, 323)

# Histogram of First Sample

In [51]:
sns.distplot(normalized_matrix.iloc[:, 10]);

# Histogram of First Gene

In [52]:
sns.distplot(normalized_matrix.iloc[10, :]);

# Save Filtered Matrix

In [53]:
filename = path+'tcga_kirp_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [54]:
gene_list = uf.createGeneList(normalized_matrix)

In [55]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [56]:
gene_list.shape

(18546, 2)

# Save Gene List

In [57]:
filename = path+'tcga_kirp_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [58]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [59]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [60]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-2K-A9WE-01,53,MALE,M0,NX,T2b,Stage II,Kidney Papillary Renal Cell Carcinoma,Primary Solid Tumor,Kidney,NaN
TCGA-2Z-A9J1-01,71,MALE,MX,N0,T1a,Stage I,Kidney Papillary Renal Cell Carcinoma,Primary Solid Tumor,Kidney,NaN
TCGA-2Z-A9J2-01,71,FEMALE,MX,NX,T1a,Stage I,Kidney Papillary Renal Cell Carcinoma,Primary Solid Tumor,Kidney,NaN
TCGA-2Z-A9J3-01,67,MALE,MX,NX,T2,Stage II,Kidney Papillary Renal Cell Carcinoma,Primary Solid Tumor,Kidney,NaN
TCGA-2Z-A9J5-01,80,MALE,MX,NX,T2,Stage II,Kidney Papillary Renal Cell Carcinoma,Primary Solid Tumor,Kidney,NaN


In [61]:
attribute_list.shape

(323, 10)

# Save Attribute List

In [62]:
filename = path+'tcga_kirp_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [63]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [64]:
standard_matrix.head()

,TCGA-2K-A9WE-01,TCGA-2Z-A9J1-01,TCGA-2Z-A9J2-01,TCGA-2Z-A9J3-01,TCGA-2Z-A9J5-01,TCGA-2Z-A9J6-01,TCGA-2Z-A9J7-01,TCGA-2Z-A9J8-01,TCGA-2Z-A9J9-01,TCGA-2Z-A9JD-01,...,TCGA-Y8-A894-01,TCGA-Y8-A895-01,TCGA-Y8-A896-01,TCGA-Y8-A897-01,TCGA-Y8-A898-01,TCGA-Y8-A8RY-01,TCGA-Y8-A8RY-11,TCGA-Y8-A8RZ-01,TCGA-Y8-A8S0-01,TCGA-Y8-A8S1-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.232228,0.436479,0.833167,-0.764362,0.368377,0.269266,-0.807713,0.374531,-0.318852,0.839363,...,-0.739632,0.888989,-0.349764,-0.943816,0.009175,-0.188902,-0.974729,0.858000,-0.931446,0.690550
A1CF,-0.388587,0.478197,0.174685,-0.041979,0.205589,0.757093,-0.258635,0.509150,-0.889831,0.168488,...,-0.153457,-0.240078,-0.623721,0.366702,0.081725,0.026088,-0.611338,0.317192,-0.370028,-0.766032
A2M,0.722015,-0.336871,-0.609193,-0.968076,0.251148,-0.473079,-0.875324,-0.485427,0.945262,-0.844380,...,-0.980429,-0.794875,0.375014,0.022059,0.418374,-0.083161,0.604252,-0.435947,-0.231754,0.889471
A2ML1,0.259916,-0.792326,-0.922245,-0.433402,-0.532403,-0.037273,0.024576,0.389986,-0.810879,-0.396284,...,0.929189,0.619181,-0.334358,-0.154958,-0.674719,0.489066,0.681162,-0.359140,0.055526,-0.977892
A3GALT2,-0.411309,0.374908,0.659932,0.591715,0.331639,0.244790,0.461686,-0.875427,-0.145153,-0.640223,...,-0.293710,0.158165,-0.169941,0.672337,-0.516487,-0.918700,-0.448450,0.951359,0.040521,0.585535


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [65]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [66]:
filename = path+'tcga_kirp_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [67]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [68]:
tertiary_matrix.head()

,TCGA-2K-A9WE-01,TCGA-2Z-A9J1-01,TCGA-2Z-A9J2-01,TCGA-2Z-A9J3-01,TCGA-2Z-A9J5-01,TCGA-2Z-A9J6-01,TCGA-2Z-A9J7-01,TCGA-2Z-A9J8-01,TCGA-2Z-A9J9-01,TCGA-2Z-A9JD-01,...,TCGA-Y8-A894-01,TCGA-Y8-A895-01,TCGA-Y8-A896-01,TCGA-Y8-A897-01,TCGA-Y8-A898-01,TCGA-Y8-A8RY-01,TCGA-Y8-A8RY-11,TCGA-Y8-A8RZ-01,TCGA-Y8-A8S0-01,TCGA-Y8-A8S1-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,-1,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# Save Teriary Matrix

In [69]:
filename = path+'tcga_kirp_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [70]:
name = 'tcga_kirp_gene_up_set'

In [71]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [72]:
name = 'tcga_kirp_gene_down_set'

In [73]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [74]:
name = 'tcga_kirp_attribute_up_set'

In [75]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [76]:
name = 'tcga_kirp_attribute_down_set'

In [77]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity matrix

In [78]:
attribute_normalized_matrix = normalized_matrix.copy()

In [79]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [80]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [81]:
attribute_similarity_matix.head()

,"(Donor:TCGA-2K-A9WE, Age:53.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)","(Donor:TCGA-2Z-A9J1, Age:71.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:nan, Gradenan)","(Donor:TCGA-2Z-A9J2, Age:71.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)","(Donor:TCGA-2Z-A9J3, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)","(Donor:TCGA-2Z-A9J5, Age:80.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)","(Donor:TCGA-2Z-A9J6, Age:60.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 1, Gradenan)","(Donor:TCGA-2Z-A9J7, Age:63.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:nan, Gradenan)","(Donor:TCGA-2Z-A9J8, Age:50.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)","(Donor:TCGA-2Z-A9J9, Age:77.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 1, Gradenan)","(Donor:TCGA-2Z-A9JD, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)",...,"(Donor:TCGA-Y8-A894, Age:57.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 1, Gradenan)","(Donor:TCGA-Y8-A895, Age:74.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 1, Gradenan)","(Donor:TCGA-Y8-A896, Age:62.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:nan, Gradenan)","(Donor:TCGA-Y8-A897, Age:68.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)","(Donor:TCGA-Y8-A898, Age:69.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 1, Gradenan)","(Donor:TCGA-Y8-A8RY, Age:63.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 1, Gradenan)","(Donor:TCGA-Y8-A8RY, Age:63.0, Gender:MALE, SampleType:Solid Tissue Normal, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 1, Gradenan)","(Donor:TCGA-Y8-A8RZ, Age:55.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)","(Donor:TCGA-Y8-A8S0, Age:58.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 1, Gradenan)","(Donor:TCGA-Y8-A8S1, Age:61.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-2K-A9WE, Age:53.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Kidney Papillary Renal Cell Carcinoma, Tissue:Kidney, Tumor:Type 2, Gradenan)",1.000000,0.074413,-0.021320,-0.025804,0.154095,0.016480,-0.063671,0.004944,0.046966,-0.093390,...,-0.116516,0.096444,0.208733,0.109914,0.154368,0.093986,-0.012810,0.031

# Save Attribute Similarity Matrix

In [82]:
filename = path+'tcga_kirp_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [83]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

# Create Gene Similarity Matrix

In [84]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [85]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.061586,-0.007076,-0.153120,0.069369,-0.029990,-0.168044,0.056914,0.058666,0.132894,...,-0.042180,0.160463,-0.054529,-0.099316,-0.090169,-0.053154,-0.083436,-0.093001,-0.038713,-0.056055
A1CF,-0.061586,1.000000,0.114605,-0.110283,-0.039647,-0.122088,-0.012731,0.076841,0.135845,-0.079355,...,-0.217104,-0.069762,0.178650,-0.069571,0.249828,0.404641,0.254417,-0.120327,0.127852,0.103221
A2M,-0.007076,0.114605,1.000000,-0.195774,-0.179835,0.012286,-0.253405,-0.268028,0.167859,-0.221176,...,-0.263300,-0.128694,0.128631,0.108615,0.193891,0.099234,0.255061,-0.129445,0.074145,0.301620
A2ML1,-0.153120,-0.110283,-0.195774,1.000000,0.008146,-0.050139,0.275532,0.014026,-0.106918,-0.048683,...,0.150062,-0.116655,0.027069,0.108008,-0.045757,-0.018328,0.168772,0.060975,-0.018477,0.134386
A3GALT2,0.069369,-0.039647,-0.179835,0.008146,1.000000,0.018392,0.020206,0.100087,-0.106283,0.220171,...,0.098181,0.075851,-0.070244,-0.117716,-0.145847,-0.069948,-0.134366,0.166002,-0.072641,-0.116785


# Save Gene Similarity Matrix

In [86]:
filename = path+'tcga_kirp_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [87]:
name = 'tcga_kirp_gene_attribute_edge_list'

In [88]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  323 Out of 323   

 The number of statisticaly relevent gene-attribute associations is: 299781
